In [1]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch
import numpy as np

In [2]:
# 手写数字训练数据集
mnist = np.load("../../../../Other/datasets/NumPy/mnist.npz")
X_train, y_train, X_test, y_test = torch.unsqueeze(torch.tensor(mnist['x_train']), 1).to(dtype=torch.float32), \
                                   torch.tensor(mnist['y_train']).to(dtype=torch.long), \
                                   torch.unsqueeze(torch.tensor(mnist['x_test']), 1).to(dtype=torch.float32), \
                                   torch.tensor(mnist['y_test']).to(dtype=torch.long)

In [3]:
batch_size = 200
train_data = Data.TensorDataset(X_train, y_train)
train_loader = Data.DataLoader(dataset=train_data,
                               batch_size=batch_size,
                               shuffle=True)
train_X = Data.DataLoader(X_train, batch_size=batch_size)
test_X = Data.DataLoader(X_test, batch_size=batch_size)

In [5]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.featrues = nn.Sequential(  # 内部实现了forward函数;各模块顺序执行
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),  # 10分类问题
        )

    def forward(self, x):
        x = self.featrues(x)
        x = x.reshape(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [6]:
learning_rate = 0.01
epochs = 5
criteon = nn.CrossEntropyLoss()  # 定义损失函数为交叉熵
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = LeNet()
lenet = net.to(device)  # 模型设备转移必须在优化器定义前执行

# 多参数组的SGD优化器
optimizer = optim.SGD([dict(params=net.featrues.parameters(), momentum=0.9),
                       dict(params=net.classifier.parameters(),
                            momentum=0.95,  # 动量参数\alpha;默认momentum=0
                            nesterov=True)],  # 是否启用Nesterov动量
                      lr=learning_rate)  # 学习率

In [7]:
from torch_model.train_evaluate import Train_Evaluate

t_and_v = Train_Evaluate(model=lenet, optimizer=optimizer, criterion=criteon, epochs=5)

In [9]:
metric_reslut = t_and_v.train_eval(train_loader=train_loader, valid_sets=[(train_X, y_train), (test_X, y_test)])
metric_reslut  # 每一轮验证数据集的损失值



Train Epoch: 0  [0    /60000 (0  %)]	Loss: 0.086389
Train Epoch: 0  [2000 /60000 (3  %)]	Loss: 0.058056
Train Epoch: 0  [4000 /60000 (7  %)]	Loss: 0.096914
Train Epoch: 0  [6000 /60000 (10 %)]	Loss: 0.119162
Train Epoch: 0  [8000 /60000 (13 %)]	Loss: 0.025495
Train Epoch: 0  [10000/60000 (17 %)]	Loss: 0.047373
Train Epoch: 0  [12000/60000 (20 %)]	Loss: 0.057710
Train Epoch: 0  [14000/60000 (23 %)]	Loss: 0.107245
Train Epoch: 0  [16000/60000 (27 %)]	Loss: 0.026530
Train Epoch: 0  [18000/60000 (30 %)]	Loss: 0.051508
Train Epoch: 0  [20000/60000 (33 %)]	Loss: 0.022600
Train Epoch: 0  [22000/60000 (37 %)]	Loss: 0.043644
Train Epoch: 0  [24000/60000 (40 %)]	Loss: 0.056189
Train Epoch: 0  [26000/60000 (43 %)]	Loss: 0.071068
Train Epoch: 0  [28000/60000 (47 %)]	Loss: 0.060058
Train Epoch: 0  [30000/60000 (50 %)]	Loss: 0.053159
Train Epoch: 0  [32000/60000 (53 %)]	Loss: 0.101320
Train Epoch: 0  [34000/60000 (57 %)]	Loss: 0.091254
Train Epoch: 0  [36000/60000 (60 %)]	Loss: 0.039646
Train Epoch:

[[0.09028316289186478, 0.10863973200321198],
 [0.09001655131578445, 0.10880725085735321],
 [0.05875048041343689, 0.0808248519897461],
 [0.054144714027643204, 0.07462306320667267],
 [0.04376066476106644, 0.06450728327035904]]